In [1]:
# %pylab

import numpy
import matplotlib
from matplotlib import pyplot
np = numpy
plt = pyplot
from pylab import *
from numpy import *
%matplotlib

Using matplotlib backend: TkAgg


In [2]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [4]:
matplotlib.rcParams['figure.figsize'] = [10.5,7]
matplotlib.rcParams.update({'font.size': 25})

In [5]:
%run my_functions

# Load data file names
I load the file names of data. You have to write the data directory in `file_path` then the program sorts the file names into several categories, according to the their SimuType, and their nature.

### SimuType
- `Langevin` is the first initialisation of the cloud with the Langevin program.
- `SimuTypeQ` is the first part with velocity-Verlet. Introduces laser cooling and in a second time a ramp on the RF voltage.
- `SimuType21` is the second part. Laser cooling and trapping parameters remains constant.

### Nature of files
- `xva` contain a snapshot of the cloud : positions, velocities and accelerations of each ion at a given time, at the end of each SimuType, under the format `.bin`. For `SimuType4`, files with `xva` and `.dat` extension contain information about the Giant Molecule.
- `Temp` contain temperature of cloud as a function of time, under the form of a `.dat`.
- `trj` contain trajectory as a function of time : `trj` alone is for one or many Ca+ ions, `trjGMol` is the trajectory of the GMol.

In [51]:
# for i,j in enumerate(os.walk(file_path)):
#     print(i,j)
file_path = '/home/adrien/RemoteFS/Rivendel/Simulations/Dynamique_moleculaire/Laser_cool'+'/DC03_RF08'+'/Try00'
# print(os.listdir(file_path))

file_name_Lan = sort([x for x in os.listdir(file_path) if 'Langevin' in x or '3D' in x])
print(file_name_Lan)
file_name_SimuTypeQ = sort([x for x in os.listdir(file_path) if 'SimuTypeQ' in x])
print(file_name_SimuTypeQ)
file_name_SimuType21_xva = sort([x for x in os.listdir(file_path) if 'SimuType21' in x and 'xva' in x])
print(file_name_SimuType21_xva)

file_name_SimuTypeQ_Temp = sort([x for x in os.listdir(file_path) if 'SimuTypeQ' in x and 'Temp' in x])
print(file_name_SimuTypeQ_Temp)
file_name_SimuType21_Temp = sort([x for x in os.listdir(file_path) if 'SimuType21' in x and 'Temp' in x])
print(file_name_SimuType21_Temp)

['Langevin_cooling.info'
 'Temp_3D_Harmo_N00064_T500uK_F0.15D-19Kg_sRFG.bin'
 'xva_3D_Harmo_N00064_T500uK_F0.15D-19Kg_sRFG.bin']
['Temp_SimuTypeQ_N00064_Vrf0053_Udc0.1000D+01V_D1.0_S1.0RFG.dat'
 'xva_SimuTypeQ_N00064_Vrf0053_Udc0.1000D+01V_D1.0_S1.0RFG.bin'
 'xva_SimuTypeQ_N00064_Vrf0053_Udc0.1000D+01V_D1.0_S1.0RFG.info']
['xva_SimuType21_01_N00064_Vrf0053_Udc0.1000D+01V_D1.0_S1.0RFG_Q01_00eV.bin'
 'xva_SimuType21_01_N00064_Vrf0053_Udc0.1000D+01V_D1.0_S1.0RFG_Q01_00eV.info']
['Temp_SimuTypeQ_N00064_Vrf0053_Udc0.1000D+01V_D1.0_S1.0RFG.dat']
['Temp_SimuType21_01_N00064_Vrf0053_Udc0.1000D+01V_D1.0_S1.0RFG_Q01_00eV.dat']


# Reading snapshots

In [53]:
QGMol = 'Q01'
xva_Langevin  = load_xva_init_bin_Lan(file_path+'/'+sort(file_name_Lan)[2].strip('.bin'),1)[5]
xva_SimuTypeQ = load_xyz_init_bin_DP(file_path+'/'+sort(file_name_SimuTypeQ)[1].strip('.bin'))[0]
xva_SimuType21 = load_xyz_init_bin_DP(file_path+'/'+sort([x for x in file_name_SimuType21_xva if QGMol in x])[0].strip('.bin'))[0]

In [61]:
to_plot = xva_SimuTypeQ
figure('xva'); clf()
# title(fig_title)
subplot(211,aspect=1.0)
plot(to_plot [0,:]*1e3,to_plot [1,:]*1e3,'8',color='xkcd:purplish blue')
xlabel('x[µm]')
ylabel('y[µm]')
grid()

# subplot(212,aspect=1.0)
subplot(212)
plot(to_plot [2,:]*1e3,to_plot [0,:]*1e3,'8',color='xkcd:purplish blue')
xlabel('z[µm]')
ylabel('x[µm]')
grid()

tight_layout()

# Reading temperature

In [55]:
QGMol = 'Q01'
Temp_Langevin  = load_Temp_init_bin_Lan(file_path+'/'+sort(file_name_Lan)[1].strip('.bin'),1)[5]
Temp_SimuTypeQ = load_T_and_PM_simu(file_path+'/'+sort(file_name_SimuTypeQ)[0].strip('.dat'))
Temp_SimuType21 = load_T_and_PM_simu(file_path+'/'+sort([x for x in file_name_SimuType21_Temp if QGMol in x])[0].strip('.dat'))

[4] [4] [8] [8] [8] [55440]
N_ions 64
j_save 990
dt_j_save_next 5e-07
eta 1.5e-20
Temp [0.0005]
[5.05000000e-07 3.49779648e-03 2.90783074e-03 6.80490343e-07
 2.16332396e+00 4.68707016e-01 6.81292751e+00]


In [56]:
figure('Temperature_Lan',clear='True')
# subplot(211)
semilogy(Temp_Langevin[:,0]*1e3,mean(Temp_Langevin[:,4:7],axis=1))
grid()
xlabel('t [ms]')
ylabel('T [K]')
tight_layout()

In [57]:
figure('Temperature',clear='True')
# subplot(211)
semilogy(Temp_Langevin[:,0]*1e3,mean(Temp_Langevin[:,4:7],axis=1))
semilogy(Temp_SimuTypeQ[0]*1e3,mean(Temp_SimuTypeQ[2],axis=1))
semilogy(Temp_SimuType21[0]*1e3,mean(Temp_SimuType21[2],axis=1))
grid()
xlabel('t [ms]')
ylabel('T [K]')
tight_layout()